<a href="https://colab.research.google.com/github/zoebatz/CAP6671_project/blob/main/binary_finetuning_on_DSDSMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fine-tune BERT, MentalBERT and RoBERTa on modified DSDSMT train for binary classification

In [ ]:
!pip install transformers torch scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.1 MB/s eta 0:00:00


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import torch
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from evaluate import load
import numpy as np
import torch.nn as nn


In [ ]:
!pip install wandb

In [ ]:
import wandb
import os

from google.colab import userdata
os.environ['WANDB_API_KEY'] = userdata.get('WANDB_API_KEY')

wandb.login()

wandb: Currently logged in as: zoe-batz99 (zoe-batz99-university-of-central-florida) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
def load_data(data_name):
  # load dataset to use for fine tuning
  file_name = data_name
  path = '/content/drive/MyDrive/Colab Notebooks/'+file_name+'.csv'
  data = pd.read_csv(path)
  # combine moderate and severe labels (0 and 1) and make them 1 and make not depressed = 0
  data['label'] = data['labels'].replace({0: 1, 2: 0}) # old: new
  data = data[['text', 'label']]
  # shuffle
  data = data.sample(frac=1).reset_index(drop=True)
  return data

In [ ]:
data_test['label'].value_counts()

,count
label,
1,2397
0,848


In [ ]:
# load binary_data
data_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/binary_DSDSMT/binary_data_train.csv')
data_eval = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/binary_DSDSMT/binary_data_eval.csv')
data_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/binary_DSDSMT/binary_data_test.csv')

In [ ]:
data_train['label'].value_counts()

,count
label,
1,3751
0,2255


In [ ]:
data_eval['label'].value_counts()

,count
label,
1,600
0,400


In [ ]:
data_test['label'].value_counts()

,count
label,
1,2397
0,848


In [ ]:
# convert to hugging face dataset
train_dataset = Dataset.from_pandas(data_train)
eval_dataset = Dataset.from_pandas(data_eval)
test_dataset = Dataset.from_pandas(data_test)

In [ ]:
model_name = "mental/mental-bert-base-uncased"
# "rafalposwiata/deproberta-large-v1" # DepRoBERTa from paper (TOO BIG)
#   "FacebookAI/roberta-base" # used in DSDSMT paper
#  "bert-base-uncased" # basic model
 # "mental/mental-bert-base-uncased" # model fine tuned on mental health data

# load pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Remove unneeded columns and set PyTorch format
train_dataset = train_dataset.remove_columns(["text"])
eval_dataset = eval_dataset.remove_columns(["text"])
test_dataset = test_dataset.remove_columns(["text"])

train_dataset.set_format("torch")
eval_dataset.set_format("torch")
test_dataset.set_format("torch")

Map:   0%|          | 0/6006 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3245 [00:00<?, ? examples/s]

In [ ]:
# load pretrained base model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # changed num_labels



config.json:   0%|          | 0.00/639 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at mental/mental-bert-base-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_metrics(pred):
    true_labels = pred.label_ids
    predictions = np.argmax(pred.predictions, axis=1)

    # Calculate metrics
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    #conf_matrix = confusion_matrix(true_labels, predictions)

    # Convert conf_matrix to list for JSON serialization
    #conf_matrix_list = conf_matrix.tolist()

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1,}# , true_labels, predictions  # Return labels & preds

'''
    # Print metrics
    print("Evaluation Metrics:")

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print("Confusion Matrix:")
    print(conf_matrix)
 '''

 #   acc = accuracy_score(labels, preds)
 #   precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")



'\n    # Print metrics\n    print("Evaluation Metrics:")\n\n    print(f"Accuracy: {accuracy:.4f}")\n    print(f"Precision: {precision:.4f}")\n    print(f"Recall: {recall:.4f}")\n    print(f"F1-Score: {f1:.4f}")\n    print("Confusion Matrix:")\n    print(conf_matrix)\n '

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-6,   # same as paper
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=5,  # Adjust epochs as needed
    weight_decay=0.1,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1", # better for imbalanced classes
    greater_is_better=True,   # higher f1 is better
    report_to="wandb"  # Enables logging to Weights & Biases
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-59-929fcd1d3541>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.556800,0.531319,0.743000,0.720154,0.935000,0.813633
2,0.490900,0.562171,0.727000,0.699634,0.955000,0.807611
3,0.481400,0.569821,0.727000,0.717131,0.900000,0.798226
4,0.442500,0.581986,0.727000,0.718876,0.895000,0.797327
5,0.451800,0.595048,0.714000,0.716253,0.866667,0.784314


TrainOutput(global_step=1880, training_loss=0.5036543948853269, metrics={'train_runtime': 633.0502, 'train_samples_per_second': 47.437, 'train_steps_per_second': 2.97, 'total_flos': 7901224992460800.0, 'train_loss': 0.5036543948853269, 'epoch': 5.0})

In [ ]:
# run eval to get confusion matrix
eval_results = trainer.predict(eval_dataset)

# Access predictions and labels
predictions = np.argmax(eval_results.predictions, axis=1)
true_labels = eval_results.label_ids

In [ ]:
eval_results

PredictionOutput(predictions=array([[-0.71108466,  0.20624202],
       [-0.865921  ,  0.35100085],
       [-1.0076491 ,  0.80051357],
       ...,
       [ 0.3463159 , -0.9833024 ],
       [-1.243797  ,  1.019924  ],
       [-0.9521506 ,  0.8366248 ]], dtype=float32), label_ids=array([1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1

In [ ]:
# compute confusion matrix, display and save
conf_matrix = confusion_matrix(true_labels, predictions)
print("Confusion Matrix:")
print(conf_matrix)


Confusion Matrix:
[[182 218]
 [ 39 561]]


In [ ]:
# run eval to get confusion matrix
test_results = trainer.predict(test_dataset)

# Access predictions and labels
predictions = np.argmax(test_results.predictions, axis=1)
true_labels = test_results.label_ids

test_results

PredictionOutput(predictions=array([[ 0.5357694 , -0.711256  ],
       [-1.0592537 ,  0.79328686],
       [-0.7032369 ,  0.55243564],
       ...,
       [-0.40142107, -0.00979813],
       [-0.06404316,  0.11499936],
       [-1.2155775 ,  1.0796854 ]], dtype=float32), label_ids=array([0, 1, 1, ..., 1, 1, 1]), metrics={'test_loss': 0.5247049331665039, 'test_accuracy': 0.7389830508474576, 'test_precision': 0.8055993690851735, 'test_recall': 0.8523153942428036, 'test_f1': 0.828299209406041, 'test_runtime': 19.5228, 'test_samples_per_second': 166.216, 'test_steps_per_second': 5.225})

In [ ]:
# compute confusion matrix, display and save
conf_matrix = confusion_matrix(true_labels, predictions)
print("Confusion Matrix:")
print(conf_matrix)


Confusion Matrix:
[[ 355  493]
 [ 354 2043]]


In [ ]:
model_name

'mental/mental-bert-base-uncased'

In [ ]:
# save fine tuned model to drive
model.save_pretrained('/content/drive/MyDrive/Colab Notebooks/binary_fine_tuned_DSDSMT_'+model_name)
tokenizer.save_pretrained('/content/drive/MyDrive/Colab Notebooks/binary_fine_tuned_DSDSMT_'+model_name)

('/content/drive/MyDrive/Colab Notebooks/binary_fine_tuned_DSDSMT_mental/mental-bert-base-uncased/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/binary_fine_tuned_DSDSMT_mental/mental-bert-base-uncased/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/binary_fine_tuned_DSDSMT_mental/mental-bert-base-uncased/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/binary_fine_tuned_DSDSMT_mental/mental-bert-base-uncased/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/binary_fine_tuned_DSDSMT_mental/mental-bert-base-uncased/tokenizer.json')